In [1]:
from keras.activations import leaky_relu
from sklearn.model_selection import train_test_split as tts
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam, SGD, RMSprop
from tensorflow.keras.regularizers import l1_l2, l2, l1
# for images, use Conv2D, for regression, use Conv1Dr
from tensorflow.keras.layers import Input, BatchNormalization, Dense, Activation, Conv1D, Dropout, ZeroPadding1D, Concatenate, AveragePooling1D, GlobalAveragePooling1D, LeakyReLU, Flatten

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
data = pd.read_csv('drive/MyDrive/Colab Notebooks/data/telecom_churn.csv')

In [ ]:
X = data.loc[:, 'AccountWeeks':].values
#y = to_categorical(data['Churn'].values,2)
y = data['Churn'].values

# Model

## H function

In [3]:

def H(  inputs, num_filters , dropout_rate ):
    x = BatchNormalization( epsilon=eps )( inputs )
    x = Activation('relu')(x)
    x = Conv1D(num_filters, kernel_size=(1), use_bias=False , kernel_initializer='he_normal' )(x)
    x = Dropout(rate=dropout_rate )(x)
    return x


## Transition Layer

In [4]:
def transition(inputs, num_filters , compression_factor , dropout_rate ):
    # compression_factor is the 'θ'
    x = BatchNormalization( epsilon=eps )(inputs)
    x = Activation('relu')(x)
    num_feature_maps = inputs.shape[1] # The value of 'm'

    x = Conv1D( np.floor( compression_factor * num_feature_maps ).astype(int) ,
                               kernel_size=(1), use_bias=False, padding='same' , kernel_initializer='he_normal' , kernel_regularizer=l2( 1e-4 ) )(x)
    x = Dropout(rate=dropout_rate)(x)
    
    # x = AveragePooling2D(pool_size=(2, 2))(x)
    return x


## Dense Block

In [5]:
def dense_block( inputs, num_layers, num_filters, growth_rate , dropout_rate ):
    for i in range(num_layers): # num_layers is the value of 'l'
        conv_outputs = H(inputs, num_filters , dropout_rate )
        inputs = Concatenate()([conv_outputs, inputs])
        num_filters += growth_rate # To increase the number of filters for each layer.
    return inputs, num_filters


## Initialize Model Inputs and Outputs

In [6]:
input_shape = ( 10, 1 ) 
num_blocks = 4
num_layers_per_block = 4
growth_rate = 2
dropout_rate = 0.2
compress_factor = 0.5
eps = 1.1e-5

num_filters = 16

inputs = Input( shape=input_shape )
x = Conv1D( num_filters , kernel_size=( 3 ) , use_bias=False, kernel_initializer='he_normal' , kernel_regularizer=l2( 1e-4 ) )( inputs )

for i in range( num_blocks ):
    x, num_filters = dense_block( x, num_layers_per_block , num_filters, growth_rate , dropout_rate )
    x = transition(x, num_filters , compress_factor , dropout_rate )

x = Flatten()(x)
# x = Dense(16,activation='gelu',use_bias=True,kernel_regularizer=l1_l2(l1=0.001,l2=0.01))(x)
# maybe flatten and add a "thinking" layer like gelu
# x = GlobalAveragePooling2D()( x ) 

# x = Dense(2)(x)
# outputs = Activation('softmax')(x)

x = Dense( 1 )( x ) # sigmoid 1 softmax 2
outputs = Activation( 'sigmoid' )( x )

## Compile Model

In [7]:
model = Model( inputs , outputs )
model.compile( loss='binary_crossentropy' ,optimizer=Adam( learning_rate=0.00001 ) ,metrics=[ 'accuracy' ])
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 10, 1)]      0           []                               
                                                                                                  
 conv1d (Conv1D)                (None, 8, 16)        48          ['input_1[0][0]']                
                                                                                                  
 batch_normalization (BatchNorm  (None, 8, 16)       64          ['conv1d[0][0]']                 
 alization)                                                                                       
                                                                                                  
 activation (Activation)        (None, 8, 16)        0           ['batch_normalization[0][0]']

## Application

In [ ]:
batch_size = 16
epochs = 25

x_train, x_test, y_train, y_test = tts(X, y, test_size = 0.25, stratify=y)

model.fit( x_train, y_train, epochs=epochs , batch_size=batch_size , validation_data=( x_test , y_test ) )

Epoch 1/25
157/157 [==============================] - 21s 32ms/step - loss: 1.2111 - accuracy: 0.3189 - val_loss: 0.7675 - val_accuracy: 0.1595
Epoch 2/25
157/157 [==============================] - 5s 32ms/step - loss: 1.1082 - accuracy: 0.3593 - val_loss: 0.7538 - val_accuracy: 0.2278
Epoch 3/25
157/157 [==============================] - 6s 37ms/step - loss: 1.0301 - accuracy: 0.3870 - val_loss: 0.7292 - val_accuracy: 0.3285
Epoch 4/25
157/157 [==============================] - 4s 28ms/step - loss: 0.9307 - accuracy: 0.4398 - val_loss: 0.6978 - val_accuracy: 0.4988
Epoch 5/25
157/157 [==============================] - 6s 36ms/step - loss: 0.8749 - accuracy: 0.4630 - val_loss: 0.6629 - val_accuracy: 0.7422
Epoch 6/25
157/157 [==============================] - 5s 33ms/step - loss: 0.8051 - accuracy: 0.5118 - val_loss: 0.6490 - val_accuracy: 0.7878
Epoch 7/25
157/157 [==============================] - 4s 28ms/step - loss: 0.7417 - accuracy: 0.5726 - val_loss: 0.6224 - val_accuracy: 0.835

Softmax Results

In [ ]:
results = model.evaluate(x_test, y_test, batch_size=batch_size)
print( 'Loss = {} and Accuracy = {} %'.format( results[0] , results[1] * 100 ) )

53/53 [==============================] - 0s 5ms/step - loss: 0.4662 - accuracy: 0.8549
Loss = 0.46621909737586975 and Accuracy = 85.49160957336426 %


Sigmoid Results

In [ ]:
results = model.evaluate(x_test, y_test, batch_size=batch_size)
print( 'Loss = {} and Accuracy = {} %'.format( results[0] , results[1] * 100 ) )

53/53 [==============================] - 1s 10ms/step - loss: 0.4412 - accuracy: 0.8549
Loss = 0.44123995304107666 and Accuracy = 85.49160957336426 %
